In [82]:
import threading
import time

In [83]:
#Install these libs if you don't have one
# pip install PySimpleGUI
# pip install tkhtmlview

In [95]:
import PySimpleGUI as sg
from tkhtmlview import html_parser
import numpy as np
import webbrowser

In [96]:
html_parser = html_parser.HTMLTextParser()
count_thereshold = 60


In [97]:
def open_site(link_address):
    file = webbrowser.open(link_address)
    print('You entered ', link_address)


In [98]:
def SetWidget(wid, htmltxt):
    set_html(wid, htmltxt)
    width1, height1 = wid.winfo_width(), wid.winfo_height()

In [99]:
exitFlag = 0

def countdown(threadName, link_address, wid,whichChanel ):
    start_time = time.time()
    counter = 0
    t = count_thereshold
    
    while t > 0 :
        if whichChanel == "Chanel #1" and isWeb1Occupied == False:
            break
        elif whichChanel == "Chanel #2" and isWeb2Occupied == False:
            break
            
        t -= 1
        html = f"""
<div class="infobox-caption" style="text-align:center">{whichChanel} displays <br>{link_address}</div>
<div class="infobox-caption" style="text-align:center">Wait for <br> {t} </br>seconds</div>
"""
        SetWidget(wid, html)
        time.sleep(1)
        
        if isWeb1Occupied == False:
            return
        elif t <= 0:
            t = count_thereshold
    
    print(f"time: {t}: {time.time()-start_time} and counter: {counter}");
    
    print(f"{link_address} is out of time. Now reloading...\n")
    # we can also use count_thread.is_alive()

In [100]:
def set_html(widget, html, strip=True):
    prev_state = widget.cget('state')
    widget.config(state=sg.tk.NORMAL)
    widget.delete('1.0', sg.tk.END)
    widget.tag_delete(widget.tag_names)
    html_parser.w_set_html(widget, html, strip=strip)
    widget.config(state=prev_state)

In [101]:
class WebThread (threading.Thread):
    def __init__(self, url, wid, chanelStr, chanelID):
        threading.Thread.__init__(self)
        self.url = url
        self.wid = wid
        self.chanelID = chanelID
        self.name = chanelStr
        self.terminate = False
        self.html = f"""
<div class="infobox-caption" style="text-align:center">{self.name} displays <br>{self.url}</div>
"""
        SetWidget(self.wid, self.url)
        
    def run(self): # call when created
        print ("Starting " + self.name + "\n")
        print(f"{self.chanelID-1}")
        open_site(self.url)
        countdown(self, self.url, self.wid, self.name) #
        print("Exiting " + self.name + "\n")    
    


In [102]:
isWeb1Occupied = False
isWeb2Occupied = False

In [103]:
def alicia_project():
    global isWeb1Occupied
    isWeb1Occupied = False
    global isWeb2Occupied
    isWeb2Occupied = False
    url = "www.default.com"
    
    
    layout_web1 = [
        [sg.Multiline(
        size=(15, 10),
        border_width=2,
        text_color='blue',
        background_color='white',
        disabled=True,
        no_scrollbar=True,
        expand_x=True,
        expand_y=True,
        key='Web1')], ]
    
    layout_web2 = [
        [sg.Multiline(
        size=(15, 10),
        border_width=2,
        text_color='blue',
        background_color='white',
        disabled=True,
        no_scrollbar=True,
        expand_x=True,
        expand_y=True,
        key='Web2')], ]
    
    layout = [[sg.Text("Put some URL here:")],
            [sg.InputText(key="-URL-")],
            [sg.Frame("Web1",  layout_web1, expand_x=True, expand_y=True)],
            [sg.Frame("Web2",  layout_web2, expand_x=True, expand_y=True)],
            [sg.Button('Display'), sg.Exit()]]
    

    def Exit():
        exitFlag = 1
        if thread1.is_alive() and thread2.is_alive():
            thread1.join()
            thread2.join()
        window.close()
        return

    # Create the window
    window = sg.Window("Concurrent Project", layout, finalize=True, use_default_focus=False)
    

    
    # Create an event loop
    while True:
        event, values = window.read()
        if event in (sg.WIN_CLOSED, 'Exit') or event == sg.WIN_CLOSED:
            break
        elif event == "Display":
            
            url = values["-URL-"]
            if isWeb1Occupied == True and  isWeb2Occupied == True:
                thread1.terminate = True
                thread2.terminate = True
                isWeb1Occupied = False
                isWeb2Occupied = False
                time.sleep(2)
                print("Web1 and Web2 were full. Now we reset.")
#                 Exit()
#            we have to remove thread also
            if isWeb1Occupied:
                thread2 = WebThread(url, window['Web2'].Widget, "Chanel #2",2)
                thread2.start()
                
                isWeb2Occupied = True
            else:
                thread1 = WebThread(url, window['Web1'].Widget, "Chanel #1",1)
                thread1.start()
                isWeb1Occupied = True
    Exit()


In [104]:
if __name__ == '__main__':
    
    alicia_project()
    print('Exiting Program')


Starting Chanel #1

0
You entered  sanook.com
Starting Chanel #2

1
You entered  finland.com
Exiting Chanel #2

Exiting Chanel #1

Web1 and Web2 were full. Now we reset.
Starting Chanel #1

0
You entered  facebook.com
Exiting Program


Exception in thread Chanel #1:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\preaw\AppData\Local\Temp\ipykernel_35816\1025586392.py", line 18, in run
  File "C:\Users\preaw\AppData\Local\Temp\ipykernel_35816\986413862.py", line 19, in countdown
  File "C:\Users\preaw\AppData\Local\Temp\ipykernel_35816\1064178219.py", line 2, in SetWidget
  File "C:\Users\preaw\AppData\Local\Temp\ipykernel_35816\2038574618.py", line 2, in set_html
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1652, in cget
    return self.tk.call(self._w, 'cget', '-' + key)
RuntimeError: main thread is not in main loop


## References

* PySimpleGUI: https://www.pysimplegui.org/en/latest/cookbook/
* Countdown: https://stackoverflow.com/questions/47558622/how-do-i-make-a-count-down-timer-to-run-alongside-my-game-in-python
* Multithread Example: https://www.darkartistry.com/how-to-create-a-pysimplegui-program/